## Exploration of atlas for RAG

### Import statements and dependencies

In [2]:
import dotenv
import os
from bson import ObjectId
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_transformers.openai_functions import create_metadata_tagger
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pymongo import MongoClient
from rich import print as rprint

dotenv.load_dotenv()

True

### Databse Ping

In [34]:
MONGODB_URI = os.getenv("MONGODB_URI")

try:
    client = MongoClient(MONGODB_URI)
    db = client["atlasexploration"]
    print("Connection successful!")
except Exception as e:
    print("Connection failed:", e)

Connection successful!


### Databse Setup

In [7]:
client = MongoClient(os.getenv("MONGODB_URI"))

dbName = "atlas_exploration"
collectionName = "chunked_data"
index = "vector_index"
collection = client[dbName][collectionName]

### Loading the document

In [19]:
loader = PyPDFLoader("../data/M9A_3rd_Edition.pdf")
pages = loader.load()
rprint(pages[3])

Document(
    metadata={'source': '../data/M9A_3rd_Edition.pdf', 'page': 3},
    page_content=' Table Of Cont ents \nCopyright reserved by the Singapore College of Ins urance Limited [Version 
1. 0] i \nContents  Chapter 1 Introduction To Structured Products  1 \n \n1. What Is A Structured Product?  \nA. 
How Does A Structured Product Work?  \nB. What Is A Wrapper?  \nC. Why Invest In Structured Products?  \nD. 
Challenges Facing Structured Products  \n2. Components Of A Structured Product  \nA. Principal Risk Versus Return 
Risk  \nB. Trade- off Between Risk And Return  \n3. Types Of Structured Products  \nA. Products Designed To Protect
Capital  \nB. Yield Enhancement Products  \nC. Participation Products  \n4. Similarities And Differences Of 
Structured Products  \nA. Similarities In Features  \nB. Differences In Features  \nC. Differences In Rights  \nD. 
Similarities And Differences In Governance  \n5. Suitability  \nA. Know Your Clients  \nB. Know Your Products  \n 
\n \nChapter 2 Risk Considerations Of Structured Products  27 \n 1. Market Risk  \nA.  General Market Risk  \nB.   
Issuer -specific Risk  \n2. Issuer Or Swap Counterparty Credit Risk  \n3. Liquidity Risk  \n4. Foreign Exchange 
(FX) Risk  \n5. Structural Risk  \nA. Safety Of Principal  \nB. Leverage \nC. Investment In Derivatives  \nD. 
Investment Concentration \nE. Collateral  \n6. Other Risks  \nA. Legal And Regulatory  \nB. Correlation  \nC. 
Modelling  \nD. Early Redemption  \nE. Examples Of Redemption Amount  \n \n \n \n  '
)

### Sanitization of data

In [10]:
# removing pages with too little content

cleaned_pages = []
for page in pages:
    if len(page.page_content.split(" ")) > 20:
        cleaned_pages.append(page)

rprint(len(cleaned_pages))
rprint(cleaned_pages[1])

138

Document(
    metadata={'source': '../data/M9A_3rd_Edition.pdf', 'page': 2},
    page_content=' \nLIFE INSURANCE AND  \nINVESTMENT- LINKED POLICIES II  \n \n3rd Edition, Version 1.0 –  April 
2023 \n \n  \n \n  \n \n \n  \n \n  \n  \n \n \n \n \n \n \n \n \n \n \n \n© 2023  by Singapore College of 
Insurance Limited. All rights reserved.  \n \nAny reproduction or redistribution of this Study Text in part or in 
its entirety, without granted \npermission, other than for the purpose of your examination preparation, is strictly
prohibited \nand will be tantamount to a copyright infringement. Legal action will be taken to protect our 
\ncopyright. Any website references are correct as of the time of publication.  \n \nThis Study Text is designed as
a learning programme. The SCI is not engaged in rendering \nlegal, tax, investment or other professional advice and
the reader should consult professional counsel as appropriate. We have tried to provide you with the most accurate 
and  useful \ninformation possible. However, the information in this publication may be affected by changes \nin 
law or industry practice, and, as a result, information contained in this publication may \nbecome outdated. This 
material should in no way be used as an  original source of authority on \nlegal matters. Any names used in this 
Study Text  are fictitious and have no relationship to any \npersons living or dead.  \n 1\nst Edition published in
October 2011 . \n2nd Edition published in February 2020. \n '
)

## Document enhancements

In [11]:
# splitting documents 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)

schema = {
    "properties": {
        "title": {"type": "array", "items": {"type": "string"}},
        "hasCode": {"type": "boolean"},
    },
    "required": ["title", "keywords", "hasCode"],
}

llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0, model="gpt-3.5-turbo")

document_transformer = create_metadata_tagger(metadata_schema=schema, llm=llm)
docs = document_transformer.transform_documents(cleaned_pages)
split_docs = text_splitter.split_documents(docs)

rprint(len(split_docs))
rprint(split_docs[20])
rprint(split_docs[21])

927

Document(
    metadata={
        'title': ['Chapter 3 Understanding Derivatives', 'Chapter 4 Introduction To Structured ILPs'],
        'hasCode': False,
        'source': '../data/M9A_3rd_Edition.pdf',
        'page': 4
    },
    page_content='D. Bullish Option Strategies  \nE. Bearish Opt ion Strategies  \nF. Neutral Option Strategies  
\nG. Summary Of Options  \n4. Swaps  \nA. Interest Rate Swaps  \nB. Currency Swaps  \nC. Credit Default Swap (CDS) 
\nD. Equity Swaps  \nE. Commodity Swaps  \n5. Contract For Differences (CFD)  \n \n \nChapter 4 Introduction To 
Structured I LPs 73 \n \n1. What Is A Structured ILP?  \nA. Payout Under A Structured ILP  \nB. Common Terminology 
\n2. Advantages And Disadvantages Of Structured ILPs'
)

Document(
    metadata={
        'title': ['Chapter 3 Understanding Derivatives', 'Chapter 4 Introduction To Structured ILPs'],
        'hasCode': False,
        'source': '../data/M9A_3rd_Edition.pdf',
        'page': 4
    },
    page_content='1. What Is A Structured ILP?  \nA. Payout Under A Structured ILP  \nB. Common Terminology  \n2. 
Advantages And Disadvantages Of Structured ILPs  \nA. Advantages Of Investing In Structured ILPs  \nB. 
Disadvantages Of Investing In Structured ILPs  \nC. Risk Considerations For Structured ILPs  \nD. Examples Of 
Structured ILPs  \n3. Who Would Invest In Structured ILPs?  \n4. When Are Structured ILPs Unsuitable?  \n5. 
Governance  \nA. Legal Structure Of ILP  \nB. Investment Guidelines For Retail Funds'
)

### Embedding Generation & Vector Store setup

In [14]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

vectorStore = MongoDBAtlasVectorSearch.from_documents(
    split_docs, embeddings, collection=collection
)

### Vector Store Retriever and Vector Store must use the same embedding model

In [9]:
vectorStoreRetriever = MongoDBAtlasVectorSearch.from_connection_string(
    os.getenv("MONGODB_URI"),
    dbName + "." + collectionName,
    OpenAIEmbeddings(disallowed_special=(), openai_api_key=os.getenv("OPENAI_API_KEY")),
    index_name=index,
)

### Function for querying data

In [39]:
retriever = vectorStoreRetriever.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3, "pre_filter": {"hasCode": {"$eq": False}}},
    )

def query_data_similarity(query, retriever):
    """
    k: Number of results.
    """

    results = retriever.invoke(query)
    rprint(results)

    return results


def query_data_mmr(query, vectorStore):
    """
    k: Number of results.
    lambda: Balance between relevance and diversity (typically between 0 and 1).
    ----------------------------------------------------------------------------
    Retrieve more documents with higher diversity useful if your dataset has many 
    similar documents
    """

    retriever = vectorStore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "lambda": 0.5},
    )

    results = retriever.invoke(query)
    filtered_results = [{"text": result.text, "metadata": result.metadata} for result in results]

    rprint(filtered_results)

    return filtered_results


def query_data_threshold(query, vectorStore):
    """
    k: Number of results.
    alpha: Controls the weight between vector similarity and keyword relevance, with 0.5 being an equal balance.
    ------------------------------------------------------------------------------------------------------------
    Only retrieve documents that have a relevance score above a certain threshold
    """
    
    retriever = vectorStore.as_retriever(
        search_type="similarity_score_threshold",
    )

    results = retriever.invoke(query)
    rprint(results)

    return results


### Vector Search based on Similarity

In [40]:
query = "What is Collective Investment Scheme (CIS)"

query_data_similarity(query, retriever)

[
    Document(
        metadata={
            '_id': '671c7fd2d979057b1f309d21',
            'title': ['Participation Products', 'Tracker certificate', 'Bonus certificate', 'Airbag certificate'],
            'hasCode': False,
            'source': '../data/M9A_3rd_Edition.pdf',
            'page': 35
        },
        page_content='• Collective Investment Scheme (CIS) – Code on CIS (the “Code”) issued\nby the MAS.\n• 
Investment-linked Policy - Insurance Act 1966.\nSuitability of structured products •knowing your client  - his 
investment objectives, risk appetite, time\nhorizon, financial position, investment knowledge and 
experience.\n•know the products under consideration, so that the product features\nand risk factors can be 
explained to the client in a way that he canunderstand.\nInvestment objectives •\nSafety.\n•Income.'
    ),
    Document(
        metadata={
            '_id': '671c7fd2d979057b1f309cdf',
            'title': ['Module 9A: Life Insurance and Investment -Linked Policies II'],
            'hasCode': False,
            'source': '../data/M9A_3rd_Edition.pdf',
            'page': 27
        },
        page_content='Module 9A: Life Insurance and Investment -Linked Policies II  \n22 Copyright reserved by the 
Singapore College of Insurance Limited [Version 1. 0]   Market -Maker, there is no guarantee that the investor will
be able to dispose of \nhis investment at a price which he considers reasonable9. \n \nD2.        Collective 
Investment Scheme (CIS)  \n \n4.18 A CIS is a pooled investment vehicle, where individual investors invest in a 
\ncommon fund, and a professional investment manager directs the investments'
    ),
    Document(
        metadata={
            '_id': '671c7fd2d979057b1f309c6f',
            'title': ['Introduction to Structured Products', 'What Is A Wrapper?'],
            'hasCode': False,
            'source': '../data/M9A_3rd_Edition.pdf',
            'page': 10
        },
        page_content='(c) Structured Funds : These are Collective Investment Schemes (CIS), also \nknown as 
investment funds. In Singapore, most Singapore- domiciled \ninvestment funds are structured as trusts, while 
foreign -domiciled funds are \ntypically structured as corporations. In the case of a fund structured as a trust, 
there is an independent trustee making sure that the fund manager’s operation of the fund is in accordance with the
trust document and'
    )
]

[Document(metadata={'_id': '671c7fd2d979057b1f309d21', 'title': ['Participation Products', 'Tracker certificate', 'Bonus certificate', 'Airbag certificate'], 'hasCode': False, 'source': '../data/M9A_3rd_Edition.pdf', 'page': 35}, page_content='• Collective Investment Scheme (CIS) – Code on CIS (the “Code”) issued\nby the MAS.\n• Investment-linked Policy - Insurance Act 1966.\nSuitability of structured products •knowing your client  - his investment objectives, risk appetite, time\nhorizon, financial position, investment knowledge and experience.\n•know the products under consideration, so that the product features\nand risk factors can be explained to the client in a way that he canunderstand.\nInvestment objectives •\nSafety.\n•Income.'),
 Document(metadata={'_id': '671c7fd2d979057b1f309cdf', 'title': ['Module 9A: Life Insurance and Investment -Linked Policies II'], 'hasCode': False, 'source': '../data/M9A_3rd_Edition.pdf', 'page': 27}, page_content='Module 9A: Life Insurance and Inve

### Function for getting a document by id

In [35]:
def get_documents_by_ids(ids, collection):
    object_ids = [ObjectId(id) for id in ids]
    documents = collection.find({"_id": {"$in": object_ids}})
    result = [{"_id": document["_id"], "text": document["text"], "title": document["title"], "source": document["source"], "page": document["page"]} for document in documents]
    return result

ids = ["671c7fd2d979057b1f309c45", "671c7fd2d979057b1f309c46"]
documents = get_documents_by_ids(ids, collection)
rprint(documents)

[
    {
        '_id': ObjectId('671c7fd2d979057b1f309c45'),
        'text': 'B. Practical Examples Of Forward Contracts  \nC. Summary Of Forward Contracts  \nD. History Of 
Futures Contracts  \nE. Pricing Of Futures Contracts  \nF. Margin (Futures Contracts)  \nG. Market Participants  
\nH. Futures Trading Strategies  \n3. Options And Warrants  \nA. Risk Profile Of Option / Warrant  \nB. Plain 
Vanilla Versus Exotic Option  \nC. Basic Option Trading Strategies  \nD. Bullish Option Strategies  \nE. Bearish 
Opt ion Strategies  \nF. Neutral Option Strategies  \nG. Summary Of Options',
        'title': ['Chapter 3 Understanding Derivatives', 'Chapter 4 Introduction To Structured ILPs'],
        'source': '../data/M9A_3rd_Edition.pdf',
        'page': 4
    },
    {
        '_id': ObjectId('671c7fd2d979057b1f309c46'),
        'text': 'D. Bullish Option Strategies  \nE. Bearish Opt ion Strategies  \nF. Neutral Option Strategies  
\nG. Summary Of Options  \n4. Swaps  \nA. Interest Rate Swaps  \nB. Currency Swaps  \nC. Credit Default Swap (CDS) 
\nD. Equity Swaps  \nE. Commodity Swaps  \n5. Contract For Differences (CFD)  \n \n \nChapter 4 Introduction To 
Structured I LPs 73 \n \n1. What Is A Structured ILP?  \nA. Payout Under A Structured ILP  \nB. Common Terminology 
\n2. Advantages And Disadvantages Of Structured ILPs',
        'title': ['Chapter 3 Understanding Derivatives', 'Chapter 4 Introduction To Structured ILPs'],
        'source': '../data/M9A_3rd_Edition.pdf',
        'page': 4
    }
]

### Prompt Engineering

In [44]:
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Do not answer the question if there is no given context.
Do no answer the question if it is not related to the context.
Context:
{context}
Question: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)
retrieve = {
    "context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
    "question": RunnablePassthrough()
}

llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0)

response_parser = StrOutputParser()

rag_chain = (
    retrieve
    | custom_rag_prompt
    | llm
    | response_parser
)

rprint(rag_chain.invoke(query))

Collective Investment Scheme (CIS) is a pooled investment vehicle where individual investors invest in a common 
fund, and a professional investment manager directs the investments.